# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 05** - **Tópico: Máquinas de Vetores de Suporte em Problemas Multiclasse**

<br>

O algoritmo **Máquinas de Vetores de Suporte**, conhecido por **SVM** (de *Support Vector Machines*, em inglês), é amplamente utilizado para tarefas de classificação e regressão, possuindo uma grande versatilidade e alto poder de modelagem para problemas lineares e não-lineares.

Conforme discutido em aula, o algoritmo encontra a fronteira de decisão formulando a tarefa de aprendizado como um problema de otimização. O SVM localiza a reta ou hiperplano que separa as instâncias de treinamento em duas regiões de forma a maximizar a margem entre os vetores de suporte das duas classes analisadas. Para problemas multiclasse, a tarefa de classificação é decomposta em um conjunto de problemas de classificação binária.

A versatilidade do SVM vem do uso de diferentes tipos de kernel, que são funções que realizam transformações nos dados a fim de projetá-los em um espaço de dimensão superior, de forma a permitir que relações não-lineares sejam modeladas com um classificador linear. Além do kernel linear, outros tipos de kernel muito utilizados são o Polinomial e o Radial Basis Function (RBF).

<br> 

**Objetivo deste notebook**: Exercitar o uso de SVM em tarefas de **classificação multiclasse**.

<br>

---



##**Determinando a qualidade de vinhos verdes Portugueses**

*Seria possível prever as preferências humanas sobre vinhos verdes (principalmente de provadores profissionais) a partir de testes analíticos que quantificam propriedades químicas dos vinhos?*

Nesta atividade, vamos analisar um conjunto de observações sobre diversas amostras de vinho verde Português (do tipo branco), envolvendo suas propriedades químicas e classificação realizada por provadores através de análise sensorial. 

O preço do vinho depende de um conceito bastante abstrato de apreciação do vinho a partir de análise sensorial pelos provadores, cuja opinião pode ter um alto grau de variabilidade. Outro fator chave na certificação e avaliação da qualidade do vinho são os testes análiticos realizados em laboratório e facilmente disponíveis na etapa de certificação. Estes testes levam em consideração fatores como acidez, nível de pH, presença de açúcar e outras propriedades químicas. Para o setor vitivinícola, seria de interesse que a análise de qualidade humana no processo de degustação pudesse estar relacionada com as propriedades químicas do vinho aferidas com os testes analíticos. Assim,  o processo de avaliação e certificação da garantia da qualidade do vinho poderia ser mais controlado.


O conjunto de dados a ser analisado nesta atividade possui informações para 4898 amostras de vinhos verdes (tipo branco), todas produzidos em uma determinada zona de Portugal. Os atributos são coletados para 12 propriedades diferentes dos vinho: uma das quais é Qualidade, com base em análise sensorial por provadores, e as restantes são propriedades químicas dos vinhos, incluindo densidade, acidez, teor alcoólico, etc. Todas as propriedades químicas dos vinhos são variáveis ​​contínuas. A qualidade é uma variável ordinal com uma classificação possível de 1 (pior) a 10 (melhor). Cada variedade de vinho é provada por três provadores independentes e a classificação final atribuída é a classificação mediana dada pelos provadores.

O conjunto de dados original foi publicado no artigo: *Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Modeling wine preferences by data mining from physicochemical properties. Decision support systems, 47(4), 547-553.*

O objetivo desta tarefa é abordar a questão apontada no início desta seção, desenvolvendo um modelo capaz de predizer a classificação de um vinho com base nas suas propriedades químicas avaliadas em testes analíticos.



---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais
from sklearn.svm import SVC  ## para treinar um SVM
from sklearn.model_selection import train_test_split # para divisão de dados
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,ConfusionMatrixDisplay ## para avaliação dos modelos
sns.set()

In [ ]:
df = pd.read_table("https://drive.google.com/uc?export=view&id=11YsVJck74_gyADzGJSU9Uwn8cDq_l3BD",sep=";")
df.head()  # para visualizar apenas as 5 primeiras linhas

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'quality'* contém a classificação de qualidade de cada vinho, aferida pelos provadores. Vamos avaliar como as instâncias estão distribuídas entre os diferentes valores de *'quality'* presentes no dataset (originalmente os mesmos podem variar entre 0 e 10).

In [ ]:
## Distribuição do atributo alvo, 'quality'
sns.countplot(x='quality', data=df)

In [ ]:
## Imprimindo o valor exato de número de instâncias por nota
print(df.groupby('quality').size())

Vamos analisar a distribuição de valores para os atributos, os quais são todos numéricos como foi possível confirmar na célula de código anterior.

In [ ]:
## Criando vetor com nome dos atributos
features_names = df.columns.drop(['quality'])

In [ ]:
## Gerar um gráfico para cada variável numérica com a distribuição 
## de frequência. Avaliar a distribuição geral ou, opcionalmente, 
## a distribuição por classe (classificação do vinho) 

## Distribuição geral
def dist_plot(df,columns):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        g = sns.histplot(x=var, data=df)
    plt.tight_layout()

## Distribuição por classe
def dist_plot_perclass(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        sns.color_palette("pastel")
        g = sns.histplot(x=var, data=df, hue=label,palette='muted')
    plt.tight_layout()


dist_plot(df, features_names)
#dist_plot_perclass(df, features_names, 'quality')

O gráfico abaixo mostra de outra forma como variam os valores dos atributos, considerando todas as instâncias no conjunto de dados.


In [ ]:
df.drop(['quality'],axis=1).plot(figsize=(15,7))

Podemos observar que há grande variação na escala de valores entre os atributos. free/total sulfur oxide, alcohol, residual sugar são os atributos que mais se destacam no gráfico acima.

---


### Estruturando a tarefa de classificação

A análise exploratória dos dados nos mostrou que a qualidade dos vinhos no conjunto de dados disponível varia entre 3 e 9, com poucas instâncias sendo classificadas com as notas extremas. A distribuição dos dados sugere que a grande maioria dos vinhos possuem notas 5, 6 e 7. A falta de representatividade de vinhos com as demais classificações pode se apresentar como um problema para o desenvolvimento do modelo. Entretanto, ainda assim vamos tentar abordar o problema através de uma classificação multiclasse. Visto que não temos instâncias com notas 0-2 ou 10, vamos transformar o problema para as seguintes classes: 3 ou menos 4, 5, 6, 7, 8 ou mais


In [ ]:
df['quality'] = df['quality'].replace([3, 4], 0)
df['quality'] = df['quality'].replace([5], 1)
df['quality'] = df['quality'].replace([6], 2)
df['quality'] = df['quality'].replace([7], 3)
df['quality'] = df['quality'].replace([8, 9], 4)
sns.countplot(x='quality', data=df)


---


### Criando conjuntos de treino, validação e teste
 
Seguindo a mesma estratégia da aula passada, faremos a divisão dos dados em treino, validação e teste na proporção (sugerida) de 70%/15%/15% para fazer avaliação de desempenho dos modelos com um *holdout de 3 vias*. Na célula de código abaixo, os dados são inicialmente divididos entre duas variáveis: uma variável contendo os atributos e outra a classe a ser predita. Na sequência, implemente os seguintes passos:


1.   Utilize a fução train_test_split() para dividir os dados em treino, validação e teste. Caso tenha dúvidas, use como exemplo o código da aula 02.
2.   Normalize os dados usando o método MaxMin. Lembre-se que primeiramente os parâmetros de normalização são estimados a partir dos dados de treino e então aplicados a todos os três conjuntos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['quality'], axis=1)
y = df['quality'].values

In [ ]:
## Definindo as proporções de treino, validação e teste.
train_ratio = 0.75
test_ratio = 0.15
validation_ratio = 0.15

## Fazendo a primeira divisão, para separar um conjunto de teste dos demais.
## Assuma X_train e y_train para os dados de treinamento e X_test e y_test para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio,stratify=y)

## Fazendo a segunda divisão, para gerar o conjunto de treino e validação a partir 
## do conjunto de 'treinamento' da divisão anterior
## Assuma X_train e y_train para os dados de treinamento e X_valid e y_valid para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio),stratify=y_train)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
## O MinMaxScaler transformará os dados para que fiquem no intervalo [0,1]
scaler = MinMaxScaler()

## Iniciar a normalização dos dados. Primeiro fazer um 'fit' do scaler nos  
## dados de treino. Esta etapa visa "aprender" os parâmetros para normalização.

scaler.fit(X_train) 

## Aplicar a normalização nos três conjuntos de dados:

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

Com os dados normalizados, podemos melhor inspecionar a distribuição de valores por classe, considerando o problema de classificação binário. Vamos fazer a análise para o conjunto de treino, observando as medianas para cada classe.

In [ ]:
df_train_norm =  pd.DataFrame(X_train,columns=features_names)
df_ytrain =  pd.DataFrame(y_train,columns=['quality'],dtype=int)
df_train_norm =  pd.concat([df_train_norm,df_ytrain], axis=1)
ave_values = df_train_norm.groupby("quality").median()
ave_values.plot(kind="bar",figsize=(20,7))

---


### Treinamento de modelos SVM multiclasse



#### Abordagem One-vs-one
No código abaixo, vamos treinar um modelo SVM fazendo a decomposição com base na abordagem **One-vs-One**. Embora esta seja a abordagem padrão adotada pelo método SVC, vamos usar o método OneVsOneClassifier, que pode ser aplicado com outras funções de aprendizado de modelos no scikit-learn. Da documentação do scikit-learn:

> OneVsOneClassifier constructs one classifier per pair of classes. At prediction time, the class which received the most votes is selected. In the event of a tie (among two classes with an equal number of votes), it selects the class with the highest aggregate classification confidence by summing over the pair-wise classification confidence levels computed by the underlying binary classifiers


O código abaixo testa 2 tipos de kernel e otimiza o hiperparâmetro C (termo de regularização). Para cada modelo, vamos avaliar a acurácia, recall e precisão. Vamos assumir que o melhor modelo será aquele que maximiza a acurácia.

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

## Definindo um array para armazenar o desempenho de cada modelo treinado e avaliado
perf_valid_ovo = []

## Definindo valores de C a serem testados
param_grid_C = [0.1, 1, 5, 10, 50, 100] #valores para C, termo de regularização
param_grid_kernel = ['linear','rbf']

# Treinando e avaliado os modelos com cada valor de hiperparâmetro especificado
for jj in range(len(param_grid_kernel)):
  for ii in range(len(param_grid_C)):
      clf = OneVsOneClassifier(SVC(kernel=param_grid_kernel[jj],C=param_grid_C[ii], class_weight='balanced'))
      clf.fit(X_train, y_train)
      pred_i = clf.predict(X_valid)
      perf_valid_ovo.append([param_grid_kernel[jj],param_grid_C[ii],accuracy_score(y_valid, pred_i),recall_score(y_valid, pred_i,average='macro'),precision_score(y_valid, pred_i,average='macro')])

In [ ]:
perf_df_ovo = pd.DataFrame(perf_valid_ovo, columns=['kernel','C','accuracy','recall','precision'])
perf_df_ovo

In [ ]:
plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df_ovo, id_vars=['C'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='C',y='value',hue='variable',palette='muted',marker='o')


plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df_ovo, id_vars=['kernel'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='kernel',y='value',hue='variable',palette='muted',marker='o')

#### Abordagem One-vs-Rest
No código abaixo, vamos treinar um modelo SVM fazendo a decomposição com base na abordagem **One-vs-Rest**. Vamos usar o método OneVsRestClassifier, que pode ser aplicado com outras funções de aprendizado de modelos no scikit-learn. Da documentação do scikit-learn:

> The one-vs-rest strategy, also known as one-vs-all, is implemented in OneVsRestClassifier. The strategy consists in fitting one classifier per class. For each classifier, the class is fitted against all the other classes. In addition to its computational efficiency (only n_classes classifiers are needed), one advantage of this approach is its interpretability. Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier. 

O código abaixo testa novamente os 2 tipos de kernel e otimiza o hiperparâmetro C (termo de regularização). 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

## Definindo um array para armazenar o desempenho de cada modelo treinado e avaliado
perf_valid_ovr = []

## Definindo valores de C a serem testados
param_grid_C = [0.1, 1, 5, 10, 50, 100] #valores para C, termo de regularização
param_grid_kernel = ['linear','rbf']

# Treinando e avaliado os modelos com cada valor de hiperparâmetro especificado
for jj in range(len(param_grid_kernel)):
  for ii in range(len(param_grid_C)):
      clf = OneVsRestClassifier(SVC(kernel=param_grid_kernel[jj],C=param_grid_C[ii], class_weight='balanced'))
      clf.fit(X_train, y_train)
      pred_i = clf.predict(X_valid)
      perf_valid_ovr.append([param_grid_kernel[jj],param_grid_C[ii],accuracy_score(y_valid, pred_i),recall_score(y_valid, pred_i,average='macro'),precision_score(y_valid, pred_i,average='macro')])

In [ ]:
perf_df_ovr = pd.DataFrame(perf_valid_ovr, columns=['kernel','C','accuracy','recall','precision'])
perf_df_ovr

In [ ]:
plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df_ovr, id_vars=['C'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='C',y='value',hue='variable',palette='muted',marker='o')


plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df_ovr, id_vars=['kernel'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='kernel',y='value',hue='variable',palette='muted',marker='o')

### Analisando os resultados para dados de teste

Nas células a seguir, vamos aplicar os modelos gerados com as estratégias OVO e OVR nos dados de teste (X_test).

In [ ]:
X_dev = np.concatenate((X_train, X_valid), axis=0)
y_dev = np.concatenate((y_train, y_valid), axis=0)
print(X_dev.shape)
#X_dev[:2,:]
#y_dev[:10]

In [ ]:
## OVO
clf_ovo = OneVsOneClassifier(SVC(kernel='rbf',C=100, class_weight='balanced'))
clf_ovo.fit(X_dev, y_dev)
y_pred_ovo = clf_ovo.predict(X_test)



cm = confusion_matrix(y_test,y_pred_ovo,labels=clf_ovo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_ovo.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print("Accuracy: ",round(accuracy_score(y_test, y_pred_ovo),3))
print("Recall: ",round(recall_score(y_test, y_pred_ovo,average='macro'),3))
print("Precision: ",round(precision_score(y_test, y_pred_ovo,average='macro'),3))


In [ ]:
## OVR
clf_ovr = OneVsRestClassifier(SVC(kernel='rbf',C=100, class_weight='balanced'))
clf_ovr.fit(X_dev, y_dev)
y_pred_ovr = clf_ovr.predict(X_test)



cm = confusion_matrix(y_test,y_pred_ovr,labels=clf_ovr.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_ovr.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print("Accuracy: ",round(accuracy_score(y_test, y_pred_ovr),3))
print("Recall: ",round(recall_score(y_test, y_pred_ovr,average='macro'),3))
print("Precision: ",round(precision_score(y_test, y_pred_ovr,average='macro'),3))
